In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date 
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm import cross_validation
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

In [7]:
user_ori=pd.read_csv('./data/users.csv',sep=',')
favorite=pd.read_csv(r'./places_event_decouper_csv/favorite.csv',sep=';')
visit=pd.read_csv(r'./places_event_decouper_csv/visit.csv',sep=';')
place=pd.read_csv(r'./places_event_decouper_csv/place.csv',sep=';',encoding='latin-1')
review=pd.read_csv(r'./places_event_decouper_csv/review.csv',sep=';',encoding='latin-1')
fos_user=pd.read_csv(r'./places_event_decouper_csv/fos_user_user.csv',sep=',')
place_type=pd.read_csv(r'./places_event_decouper_csv/place_place_type.csv',sep=';')


C:\Users\barra\AppData\Local\Temp\ipykernel_17252\2546506629.py:6: DtypeWarning: Columns (4,6,7,8,20,21,25) have mixed types. Specify dtype option on import or set low_memory=False.
  fos_user=pd.read_csv(r'./places_event_decouper_csv/fos_user_user.csv',sep=',')


In [116]:
def valeur_manquante(databases):
    all=databases.isna().sum().sort_values(ascending=False)
    percent_na=(databases.isna().sum()/databases.isna().count())
    dtype=databases.dtypes
    rapport= pd.concat([all,percent_na,dtype], axis=1,keys=["somme valeur manquante","en %","type"])
    # print(rapport)
    return rapport

def drop_columns_na(dataframe,pourcentna):
    for i in dataframe.columns:
        pourcent=(dataframe[i].isna().sum()/dataframe[i].isna().count())
        if(pourcentna<pourcent):
            dataframe.drop(i,axis=1,inplace=True)
    return dataframe


preparation de place avec drop des columns non utile au ml et ajout du place_type qui deffinit qu'elle type de bar c'est (tres important pour la recomandation)

In [117]:
placeml = place[['id','is_closed','is_published','slug','longitude','latitude','has_offers','has_food','has_terrace']]
placeml = placeml.drop(placeml[placeml.is_closed == 1].index)
placeml = placeml.drop(placeml[placeml.is_published == 0].index)
placeml = placeml.drop(["is_closed","is_published"],axis=1)
placeml.rename(columns={'id':'place_id'},inplace=True)
placeml.merge(place_type,on='place_id',how='inner')

,place_id,slug,longitude,latitude,has_offers,has_food,has_terrace,place_type_id
0,3,au-brasseur,"7,755416","48,582573",0,1.0,1.0,1
1,4,exils,"7,748596","48,579913",1,1.0,1.0,1
2,4,exils,"7,748596","48,579913",1,1.0,1.0,5
3,4,exils,"7,748596","48,579913",1,1.0,1.0,25
4,10,cafe-atlantico,"7,757637","48,583129",0,1.0,1.0,23
...,...,...,...,...,...,...,...,...
1667,7197,irish-pub-obrian,"6,8010705","47,5102158",0,1.0,1.0,36
1668,7198,le-local-1,"6,7989483","47,5098714",0,1.0,1.0,1
1669,7198,le-local-1,"6,7989483","47,5098714",0,1.0,1.0,2
1670,7199,le-baaron,"6,7975086","47,5087609",0,1.0,1.0,1


favorite mise en forme

In [118]:
favorite =  favorite.drop(["created_at"],axis=1)
favorite = favorite.drop(['id'],axis=1)


user et merge avec leur favoris


In [119]:
userP=drop_columns_na(user,0.58)
userP=userP.drop(['created_at','updated_at'],axis=1)
userP=userP.dropna(subset=['firebase_uid'])
userP.rename(columns={'id':'user_id'},inplace=True)

In [120]:

userP.merge(favorite,on='user_id',how='inner')
# print(userP.groupby(by='user_id',axis=1))


,user_id,date_of_birth,firstname,gender,firebase_uid,place_id
0,2,NaN,Mélisande,u,Pr9h7oy8OPUQjvBMyd5UbxTLo9f1,936
1,2,NaN,Mélisande,u,Pr9h7oy8OPUQjvBMyd5UbxTLo9f1,990
2,2,NaN,Mélisande,u,Pr9h7oy8OPUQjvBMyd5UbxTLo9f1,760
3,2,NaN,Mélisande,u,Pr9h7oy8OPUQjvBMyd5UbxTLo9f1,503
4,2,NaN,Mélisande,u,Pr9h7oy8OPUQjvBMyd5UbxTLo9f1,7097
...,...,...,...,...,...,...
11994,39657,NaN,Joana,NaN,JhCuJXsBuYP60WBJgz8e2C3viqQ2,2850
11995,39657,NaN,Joana,NaN,JhCuJXsBuYP60WBJgz8e2C3viqQ2,2752
11996,39657,NaN,Joana,NaN,JhCuJXsBuYP60WBJgz8e2C3viqQ2,2867
11997,39657,NaN,Joana,NaN,JhCuJXsBuYP60WBJgz8e2C3viqQ2,3961


In [121]:
reviewP=review.drop(['comment','id','created_at','updated_at'],axis=1)
reviewP
#reviewP.to_csv('reviewP.csv',sep=';',encoding='latin-1')

,rating_service,rating_atmosphere,rating_drinks,place_id,user_id
0,5,5,5,45,3
1,3,5,5,147,36
2,5,5,5,50,36
3,1,3,1,60,36
4,5,5,5,83,36
...,...,...,...,...,...
1436,4,4,4,1748,39384
1437,5,5,5,7192,39542
1438,5,5,5,186,29149
1439,5,5,4,46,39640


In [ ]:
def create_features(dataframe, features_name, id_col_name):
    """
    Generate features that will be ready for feeding into lightfm

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe which contains features
    features_name : List
        List of feature columns name avaiable in dataframe
    id_col_name: String
        Column name which contains id of the question or
        answer that the features will map to.
        There are two possible values for this variable.
        1. questions_id_num
        2. professionals_id_num

    Returns
    -------
    Pandas Series
        A pandas series containing process features
        that are ready for feed into lightfm.
        The format of each value
        will be (user_id, ['feature_1', 'feature_2', 'feature_3'])
        Ex. -> (1, ['military', 'army', '5'])
    """

    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col_name], features))
    return features

def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping 

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    features_name : List
        List of feature columns name avaiable in dataframe. 
        
    Returns
    -------
    List of all features for mapping 
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features


def calculate_auc_score(lightfm_model, interactions_matrix, 
                        question_features, professional_features): 
    """
    Measure the ROC AUC metric for a model. 
    A perfect score is 1.0.

    Parameters
    ----------
    lightfm_model: LightFM model 
        A fitted lightfm model 
    interactions_matrix : 
        A lightfm interactions matrix 
    question_features, professional_features: 
        Lightfm features 
        
    Returns
    -------
    String containing AUC score 
    """
    score = auc_score( 
        lightfm_model, interactions_matrix, 
        item_features=question_features, 
        user_features=professional_features, 
        num_threads=4).mean()
    return score

merge 

In [6]:
df_bar_type=


SyntaxError: invalid syntax (273283891.py, line 1)